L'installation [Elasticité dynamique](https://github.com/laurentperrinet/elasticite) agit comme un filtre et génère de nouveaux espaces démultipliés, comme un empilement quasi infini d'horizons. Par principe de réflexion, la pièce absorbe l'image de l'environnement et accumule les points de vue ; le mouvement permanent requalifie continuellement ce qui est regardé et entendu. 

On va maintenant utiliser des forces elastiques pour coordonner la dynamique des lames dans la trame.


<!-- TEASER_END -->

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%%writefile ../files/2015-10-14_elasticite/scenario_grille_carre.py
#!/usr/bin/env python3
# -*- coding: utf8 -*-

"""

Dans une grille en tore (pacman) privilégie les co-linéarités à angle droit.

"""
import elasticite as el
import numpy as np
class EdgeGrid(el.EdgeGrid):
    def champ(self):
        force = np.zeros_like(self.lames[2, :])
        damp = lambda t: 0.1
        noise = lambda t: 2* np.exp((np.cos(2*np.pi*(t-0.) / 6.)-1.)/ 1.**2)
        colin_t = lambda t: -2.*np.exp((np.cos(2*np.pi*(t-4.) / 6.)-1.)/ 1.**2)
        colin_d = lambda d: np.exp(-d/.05)
        #colin_d = lambda d: -1. + 2*np.exp(-np.log((d+1.e-12)/.1)**2/2/1.5**2)
        delta_angle = self.angle_relatif()-np.pi/2.
        #delta_angle *= np.sign(delta_angle)
        force += colin_t(self.t) * np.sum(np.sin(4*delta_angle)*colin_d(self.distance(do_torus=True)), axis=1)
        force += noise(self.t)*np.pi*np.random.randn(self.N_lame)
        force -= damp(self.t) * self.lames[3, :]/self.dt
        return .1 * force

if __name__ == "__main__":
    import sys
    if len(sys.argv)>1: mode = sys.argv[1]
    else: mode = 'both'
    e = EdgeGrid(N_lame=72, mode=mode, verb=False, structure=False)
    el.main(e, name='grille_carre')

Overwriting ../files/2015-10-14_elasticite/scenario_grille_carre.py


In [3]:
%run ../files/2015-10-14_elasticite/scenario_grille_carre.py notebook

""


In [4]:
%%writefile ../files/2015-10-14_elasticite/scenario_grille_contours.py
#!/usr/bin/env python3
# -*- coding: utf8 -*-

"""

Dans une grille en tore (pacman) privilégie les co-circulrités en évitant les co-linéarités

Short-range pour donner des patterns locaux

"""
import sys
if len(sys.argv)>1: mode = sys.argv[1]
else: mode = 'both'

import elasticite as el
import numpy as np
class EdgeGrid(el.EdgeGrid):
    def champ(self):
        force = np.zeros_like(self.lames[2, :])
        noise = lambda t: .1* np.exp((np.cos(2*np.pi*(t-0.) / 6.)-1.)/ 1.5**2)
        damp = lambda t: 0.001 #* np.exp(np.cos(t / 6.) / 3.**2)
        cocir_t = lambda t: 4.*np.exp((np.cos(2*np.pi*(t-2.) / 6.)-1.)/ .5**2)
        cocir_d = lambda d: np.exp(-d/.1)
        colin_t = lambda t: 2.*np.exp((np.cos(2*np.pi*(t-4.) / 6.)-1.)/ .5**2)
        colin_d = lambda d: np.exp(-d/.05)

        VM_grad = lambda angle, sigma: -2*np.sin(2*angle)*np.exp(np.cos(2*angle)/sigma**2)

        force += colin_t(self.t) * np.sum(VM_grad(self.angle_relatif(), np.pi/2)*colin_d(self.distance(do_torus=True)), axis=1)
        force += cocir_t(self.t) * np.sum(VM_grad(self.angle_cocir(do_torus=True), np.pi/6)*cocir_d(self.distance(do_torus=True)), axis=1)
        force += noise(self.t)*np.pi*np.random.randn(self.N_lame)
        force -= damp(self.t) * self.lames[3, :]/self.dt
        return .1 * force

if __name__ == "__main__":
    import sys
    if len(sys.argv)>1: mode = sys.argv[1]
    else: mode = 'both'
    e = EdgeGrid(N_lame=72, mode=mode, verb=False, structure=False)
    #el.main(e)
    name = 'grille_v1_contours'
    el.main(e, name)

Overwriting ../files/2015-10-14_elasticite/scenario_grille_contours.py


In [5]:
%run ../files/2015-10-14_elasticite/scenario_grille_contours.py notebook

""


In [6]:
%%writefile ../files/2015-10-14_elasticite/scenario_grille_triangles.py
#!/usr/bin/env python3
# -*- coding: utf8 -*-

"""

Dans une grille en tore (pacman) privilégie les co-linéarités à angles triangulaires.

"""
import sys
if len(sys.argv)>1: mode = sys.argv[1]
else: mode = 'both'

import elasticite as el
import numpy as np
class EdgeGrid(el.EdgeGrid):
    def champ(self):
        force = np.zeros_like(self.lames[2, :])
        noise = lambda t: 2* np.exp((np.cos(2*np.pi*(t-0.) / 6.)-1.)/ 1.**2)
        damp = lambda t: 0.001 #* np.exp(np.cos(t / 6.) / 3.**2)
        colin_t = lambda t: -2.*np.exp((np.cos(2*np.pi*(t-3.) / 6.)-1.)/ 1.**2)
        colin_d = lambda d: np.exp(-d/.05) #np.exp(-np.log((d+1.e-12)/.05)**2/2/1.5)

        #delta_angle = np.mod(self.angle_relatif()-np.pi/3., 2*np.pi/3)
        delta_angle = self.angle_relatif()-np.pi/3.
        #delta_angle *= np.sign(delta_angle)
        force += colin_t(self.t) * np.sum(np.sin(6*delta_angle)*colin_d(self.distance(do_torus=True)), axis=1)
        force += noise(self.t)*np.pi*np.random.randn(self.N_lame)
        force -= damp(self.t) * self.lames[3, :]/self.dt
        return .1 * force

if __name__ == "__main__":
    import sys
    if len(sys.argv)>1: mode = sys.argv[1]
    else: mode = 'both'
    e = el.EdgeGrid(N_lame=72*4, mode=mode, verb=False, structure=False)
    #el.main(e)
    name = 'grille_v1_triangles'
    el.main(e, name)

Overwriting ../files/2015-10-14_elasticite/scenario_grille_triangles.py


In [7]:
%run ../files/2015-10-14_elasticite/scenario_grille_triangles.py notebook

""


In [9]:
import elasticite as el
e = el.EdgeGrid(mode='notebook', verb=False, structure=False)
#el.main(e)
name = 'grille_v1'
el.main(e, name)

""


In [10]:
%%writefile ../files/2015-10-14_elasticite/scenario_grille_v1_tore.py
#!/usr/bin/env python
# -*- coding: utf8 -*-

"""

Dans une grille en tore (pacman) privilégie les co-circulrités en évitant les co-linéarités

Short-range pour donner des patterns locaux

"""

import elasticite as el
import numpy as np
class EdgeGrid(el.EdgeGrid):
    def champ(self):
        force = np.zeros_like(self.lames[2, :])
        noise = lambda t: 1.* np.exp((np.cos(2*np.pi*(t-0.) / 6.)-1.)/ 1.5**2)
        damp = lambda t: 0.001 #* np.exp(np.cos(t / 6.) / 3.**2)
        cocir_t = lambda t: 1.5*np.exp((np.cos(2*np.pi*(t-2.) / 6.)-1.)/ .3**2)
        cocir_d = lambda d: np.exp(-d/.1)
        colin_t = lambda t: -2.*np.exp((np.cos(2*np.pi*(t-4.) / 6.)-1.)/ .5**2)
        colin_d = lambda d: np.exp(-d/.05)

        force += colin_t(self.t) * np.sum(np.sin(2*(self.angle_relatif()))*colin_d(self.distance(do_torus=True)), axis=1)
        force += cocir_t(self.t) * np.sum(np.sin(2*(self.angle_cocir(do_torus=True)))*cocir_d(self.distance(do_torus=True)), axis=1)
        force += noise(self.t)*np.pi*np.random.randn(self.N_lame)
        force -= damp(self.t) * self.lames[3, :]/self.dt
        speed = lambda t: .2*(1-np.exp((np.cos(2*np.pi*(t-4.) / 6.)-1.)/ .5**2))
        return speed(self.t) * force

if __name__ == "__main__":
    import sys
    if len(sys.argv)>1: mode = sys.argv[1]
    else: mode = 'both'
    e = EdgeGrid(N_lame=72*4, mode=mode, verb=False, structure=False)
    #el.main(e)
    name = 'grille_v1_tore'
    el.main(e, name)

Overwriting ../files/2015-10-14_elasticite/scenario_grille_v1_tore.py


In [11]:
%run ../files/2015-10-14_elasticite/scenario_grille_v1_tore.py notebook

""
